In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

When importing dataset, by sorting with MatchID, dataset is now timeseries dataset. Every feature is (home-away) and result is decided with score diff. We have total 50 feature differences.

In [24]:
data=pd.read_csv('Data/Dataset.csv').drop(['Unnamed: 0'],axis=1).sort_values(by=['MatchID'])
df_home=data.filter(regex='home').rename(columns=lambda x:x.replace('_home',''))
df_away=data.filter(regex='away').rename(columns=lambda x:x.replace('_away',''))
df_diff=df_home-df_away
cond=[(df_diff['Score']<0),(df_diff['Score']==0),(df_diff['Score']>0)]
val=['Away','Draw','Home']
df_diff['Result']=np.select(cond,val)
df_diff=df_diff.drop(columns=['Score'])
X=df_diff.iloc[:,:-1]
y=df_diff.iloc[:,-1:]

In [25]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

## Train,Test,Validation set split
Split train,test,validation set into 0.8,0.1,0.1

In [26]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
X_test,X_val,y_test,y_val=train_test_split(X_test,y_test,test_size=0.5)

In [27]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((2836, 50), (2836, 1), (354, 50), (354, 1), (355, 50), (355, 1))

## Feature selection

In [28]:
forest=RandomForestClassifier()
forest.fit(X_train,y_train.values.ravel())
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("{}. feature {} ({:.3f})".format(f + 1, X.columns[indices][f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=45)
plt.xlim([-1, X.shape[1]])
plt.show()

Feature ranking:
1. feature Assists (0.038)
2. feature Clean sheets (0.037)
3. feature Goals (0.032)
4. feature Big Chances Created (0.030)
5. feature Goals Conceded (0.030)
6. feature Passes (0.029)
7. feature Blocked shots (0.026)
8. feature Shots on target (0.022)
9. feature Goals with right foot (0.021)
10. feature Tackle success % (0.021)
11. feature Through balls (0.020)
12. feature Cross accuracy % (0.020)
13. feature Penalties scored (0.020)
14. feature Goals with left foot (0.019)
15. feature Hit woodwork (0.019)
16. feature Accurate long balls (0.019)
17. feature Sweeper clearances (0.019)
18. feature Shooting accuracy % (0.019)
19. feature Throw outs (0.019)
20. feature Clearances off line (0.019)
21. feature Big chances missed (0.019)
22. feature Punches (0.019)
23. feature Shots (0.019)
24. feature Headed goals (0.018)
25. feature Own goals (0.018)
26. feature Catches (0.018)
27. feature Crosses (0.018)
28. feature Interceptions (0.018)
29. feature Passes per match (0.018)

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (/Users/hokuma/miniconda/lib/python3.9/site-packages/matplotlib/backend_bases.py)

In [52]:
model = LogisticRegression()
model.max_iter=1000
model.fit(X_train, y_train)

/Users/hokuma/miniconda/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [53]:
model.score(X_test,y_test)

0.596045197740113

In [54]:
model.score(X_val,y_val)

0.5352112676056338